In [1]:
import torch
import torch.nn as nn
import numpy as np
import sys; sys.path.append('../src/')
import os
from model.cmpnts import MLP
from scipy.stats import ortho_group
import matplotlib.pyplot as plt
from math import sqrt

In [2]:
data_dir = '../data/toy'

In [3]:
def nonlinearity(dataset, epsilon=1.): # epsilon can be set to the groundtruth LD manifold's average std.
    X = dataset[:]
    _, s, _ = torch.svd(X - X.mean(0), compute_uv=False)
    return s.max() / s.min(), s.min() / epsilon, epsilon, s

In [4]:
@ torch.no_grad()
def save_dataset(X, Y, i):
    in_dim = X.shape[1]
    out_dim = Y.shape[1]
    path = os.path.join(data_dir, '{}-manifold'.format(in_dim), '{}-ambient'.format(out_dim))
    os.makedirs(path, exist_ok=True)
    
    data_path = os.path.join(path, '{}-{}_{}.npy'.format(in_dim, out_dim, i))
    report_path = os.path.join(path, '{}-{}_{}.txt'.format(in_dim, out_dim, i))
    np.save(data_path, Y.detach().numpy())
    
    _, s_x, _ = torch.svd(X - X.mean(0), compute_uv=False)
    dis, ratio, eps, s = nonlinearity(Y, epsilon=s_x.mean())
    with open(report_path, 'w') as report:
        report.write('s.max / s.min = {}\n'.format(dis))
        report.write('s.min / x.s.mean = {}\n'.format(ratio))
        report.write('x.s.mean = {}\n'.format(eps / sqrt(len(X))))
        report.write('singular values = {}'.format(s.sort()[0].flip(0) / sqrt(len(X))))

# Nonlinear Dataset

In [5]:
from dataset.toy import IsometricEmbedding
from model.flow import _RealNVP

In [6]:
def build_flow(out_dim, n=10, w=256):
    nets = lambda: nn.Sequential(
        nn.Linear(out_dim, w), nn.ReLU(), 
        nn.Linear(w, w), nn.ReLU(),
        nn.Linear(w, w), nn.ReLU(),
        nn.Linear(w, out_dim), nn.Tanh())
    nett = lambda: nn.Sequential(
        nn.Linear(out_dim, w), nn.ReLU(), 
        nn.Linear(w, w), nn.ReLU(),
        nn.Linear(w, w), nn.ReLU(),
        nn.Linear(w, out_dim))
    masks = torch.as_tensor([
        [0]*(out_dim//2) + [1]*(out_dim//2), 
        [1]*(out_dim//2) + [0]*(out_dim//2)] * n, dtype=torch.float)
    flow = _RealNVP(nets, nett, masks)
    return flow

In [12]:
from tqdm import tqdm

in_dims = [64] # [1, 2, 4, 8, 16, 32, 64]
out_dims = np.asarray([2, 4, 8, 16, 32], dtype=int)
sizes = [100000] #[100, 300, 1000, 3000, 10000, 30000, 100000]

for in_dim, size in zip(in_dims, sizes):
    for out_dim in out_dims * in_dim:
        for i in range(5):
            print('{} -> {} ({})'.format(in_dim, out_dim, i))

            X = torch.randn(size, in_dim).cuda()
            Y = IsometricEmbedding(X, out_dim=out_dim)

            flow = build_flow(out_dim, 10, 256)
            Y.embed(flow = flow.cuda(),
                    optimizer = torch.optim.Adam(flow.parameters(), lr=3e-4),
                    epochs = 100,
                    batch_size = 100,
                    lam = 10.)

            save_dataset(X.cpu(), Y[:].cpu(), i)

16 -> 32 (0)


Embedding: 100%|██████████████████████████████| 100/100 [1:16:35<00:00, 45.95s/it, isometricity=0.032, linearity=0.168]    


16 -> 32 (1)


Embedding: 100%|██████████████████████████████| 100/100 [21:17<00:00, 12.77s/it, isometricity=0.0424, linearity=0.172]


16 -> 32 (2)


...

32 -> 1024 (4)


Embedding: 100%|██████████████████████████████| 100/100 [1:12:20<00:00, 43.41s/it, isometricity=0.0616, linearity=0.107]


64 -> 128 (0)


Embedding: 100%|██████████████████████████████| 100/100 [3:58:43<00:00, 143.23s/it, isometricity=0.0865, linearity=0.238] 


RuntimeError: [enforce fail at alloc_cpu.cpp:73] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 40000000000 bytes. Error code 12 (Cannot allocate memory)